In [1]:
# Dependencies
import os 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress 

#Reading in Files
cdc_df = pd.read_csv("Resource/VSRR_Provisional_Drug_Overdose_Death_Counts_2015_2021.csv", thousands=',')
care_df = pd.read_csv("Resource/Medicare_Opioid_Prescribing_Data_2013_2019.csv", low_memory = False)
caid_df = pd.read_csv("Resource/Medicaid_Opioid_Prescribing_data_2013_2019.csv", low_memory = False)

In [2]:
# Filtering Medcaid dropping year 2013 & 2014 and filtering for all plan types
caid_year = caid_df.loc[(caid_df["Year"] != 2013) & (caid_df["Year"] != 2014)]
caid_clean = caid_year.loc[caid_year["Plan_Type"]=="All"]
# Reset the index values for the dataframe
caid_clean = caid_clean.reset_index(drop=True)
caid_clean


,Year,Geo_Lvl,Geo_Cd,Geo_Desc,Plan_Type,Tot_Opioid_Clms,Tot_Clms,Opioid_Prscrbng_Rate,Opioid_Prscrbng_Rate_5Y_Chg,Opioid_Prscrbng_Rate_1Y_Chg,LA_Tot_Opioid_Clms,LA_Opioid_Prscrbng_Rate,LA_Opioid_Prscrbng_Rate_5Y_Chg,LA_Opioid_Prscrbng_Rate_1Y_Chg
0,2019,National,NaN,National,All,21978286.0,677248025.0,3.25,-3.22,-0.44,3108845.0,14.15,6.67,4.62
1,2019,State,1.0,Alabama,All,224310.0,7230251.0,3.10,-3.23,-1.08,10308.0,4.60,0.16,0.48
2,2019,State,2.0,Alaska,All,65394.0,1434139.0,4.56,-3.33,-1.20,8268.0,12.64,0.88,-0.71
3,2019,State,4.0,Arizona,All,595206.0,14923326.0,3.99,-5.14,-0.75,44311.0,7.44,-2.79,-0.52
4,2019,State,5.0,Arkansas,All,216081.0,4999626.0,4.32,-3.12,-0.88,8129.0,3.76,-0.34,-1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2015,State,51.0,Virginia,All,583116.0,10186332.0,5.72,NaN,-0.62,42311.0,7.26,NaN,-0.26
256,2015,State,53.0,Washington,All,1136474.0,14016747.0,8.11,NaN,-0.89,106143.0,9.34,NaN,0.18
257,2015,State,54.0,West Virginia,All,520370.0,9293729.0,5.60,NaN,-1.30,25575.0,4.91,NaN,0.66
258,2015,State,55.0,Wisconsin,All,962286.0,11306189.0,8.51,NaN,-0.35,146248.0,15.20,NaN,1.04


In [3]:
# Changing column name to state for merging later
caid_clean = caid_clean.rename(columns={"Geo_Desc":"State"})

caid_clean

,Year,Geo_Lvl,Geo_Cd,State,Plan_Type,Tot_Opioid_Clms,Tot_Clms,Opioid_Prscrbng_Rate,Opioid_Prscrbng_Rate_5Y_Chg,Opioid_Prscrbng_Rate_1Y_Chg,LA_Tot_Opioid_Clms,LA_Opioid_Prscrbng_Rate,LA_Opioid_Prscrbng_Rate_5Y_Chg,LA_Opioid_Prscrbng_Rate_1Y_Chg
0,2019,National,NaN,National,All,21978286.0,677248025.0,3.25,-3.22,-0.44,3108845.0,14.15,6.67,4.62
1,2019,State,1.0,Alabama,All,224310.0,7230251.0,3.10,-3.23,-1.08,10308.0,4.60,0.16,0.48
2,2019,State,2.0,Alaska,All,65394.0,1434139.0,4.56,-3.33,-1.20,8268.0,12.64,0.88,-0.71
3,2019,State,4.0,Arizona,All,595206.0,14923326.0,3.99,-5.14,-0.75,44311.0,7.44,-2.79,-0.52
4,2019,State,5.0,Arkansas,All,216081.0,4999626.0,4.32,-3.12,-0.88,8129.0,3.76,-0.34,-1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2015,State,51.0,Virginia,All,583116.0,10186332.0,5.72,NaN,-0.62,42311.0,7.26,NaN,-0.26
256,2015,State,53.0,Washington,All,1136474.0,14016747.0,8.11,NaN,-0.89,106143.0,9.34,NaN,0.18
257,2015,State,54.0,West Virginia,All,520370.0,9293729.0,5.60,NaN,-1.30,25575.0,4.91,NaN,0.66
258,2015,State,55.0,Wisconsin,All,962286.0,11306189.0,8.51,NaN,-0.35,146248.0,15.20,NaN,1.04


In [4]:
# Filtering Medicaid Data by Breakout Type and Perscription Geographic Levels for National or State levels
care_year = care_df.loc[(care_df["Year"] != 2013) & (care_df["Year"] != 2014)]
care_clean = care_year.loc[(care_year["Breakout_Type"]=="Totals") &  (care_year["Prscrbr_Geo_Lvl"]=="National") | (care_year["Prscrbr_Geo_Lvl"]=="State")  &  (care_year["Breakout"]=="Overall") & (care_df["Prscrbr_Geo_Desc"] != 'American Samoa')& (care_df["Prscrbr_Geo_Desc"] != 'Guam') & (care_df["Prscrbr_Geo_Desc"] != 'American Samoa') & (care_df["Prscrbr_Geo_Desc"] != 'Northern Mariana Islands') & (care_df["Prscrbr_Geo_Desc"] != 'Puerto Rico') & (care_df["Prscrbr_Geo_Desc"] != 'Virgin Islands')]

# Reset the index values for the dataframe
care_clean = care_clean.reset_index(drop=True)
care_clean

,Year,Prscrbr_Geo_Lvl,Prscrbr_Geo_Cd,Prscrbr_Geo_Desc,RUCA_Cd,Breakout_Type,Breakout,Tot_Prscrbrs,Tot_Opioid_Prscrbrs,Tot_Opioid_Clms,Tot_Clms,Opioid_Prscrbng_Rate,Opioid_Prscrbng_Rate_5Y_Chg,Opioid_Prscrbng_Rate_1Y_Chg,LA_Tot_Opioid_Clms,LA_Opioid_Prscrbng_Rate,LA_Opioid_Prscrbng_Rate_5Y_Chg,LA_Opioid_Prscrbng_Rate_1Y_Chg
0,2019,National,NaN,National,NaN,Totals,Overall,1239380.0,803549.0,66138200.0,1.501140e+09,4.41,-1.32,-0.27,7290510.0,11.02,-1.73,-0.77
1,2019,State,1.0,Alabama,NaN,Totals,Overall,15575.0,9739.0,1814583.0,2.800821e+07,6.48,-1.40,-0.42,138456.0,7.63,-2.13,-0.73
2,2019,State,2.0,Alaska,NaN,Totals,Overall,2847.0,1935.0,80311.0,1.639104e+06,4.90,-2.01,-0.90,13005.0,16.19,-3.73,-2.42
3,2019,State,4.0,Arizona,NaN,Totals,Overall,25876.0,16578.0,1359955.0,2.569376e+07,5.29,-1.79,-0.33,184250.0,13.55,-2.58,-1.25
4,2019,State,5.0,Arkansas,NaN,Totals,Overall,9700.0,7258.0,998319.0,1.789430e+07,5.58,-1.08,-0.32,76135.0,7.63,-1.69,-0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2015,State,51.0,Virginia,NaN,Totals,Overall,25319.0,19755.0,1720058.0,3.055526e+07,5.63,NaN,-0.29,201649.0,11.72,NaN,0.38
256,2015,State,53.0,Washington,NaN,Totals,Overall,25351.0,19532.0,1658461.0,2.374570e+07,6.98,NaN,-0.19,281071.0,16.95,NaN,-0.20
257,2015,State,54.0,West Virginia,NaN,Totals,Overall,7064.0,5174.0,752812.0,1.288542e+07,5.84,NaN,-0.56,69188.0,9.19,NaN,0.25
258,2015,State,55.0,Wisconsin,NaN,Totals,Overall,20540.0,16177.0,1489660.0,2.539947e+07,5.86,NaN,-0.15,263191.0,17.67,NaN,-0.17


In [5]:
# Changing column name to state for merging late
care_clean = care_clean.rename(columns={"Prscrbr_Geo_Desc":"State"})
care_clean

,Year,Prscrbr_Geo_Lvl,Prscrbr_Geo_Cd,State,RUCA_Cd,Breakout_Type,Breakout,Tot_Prscrbrs,Tot_Opioid_Prscrbrs,Tot_Opioid_Clms,Tot_Clms,Opioid_Prscrbng_Rate,Opioid_Prscrbng_Rate_5Y_Chg,Opioid_Prscrbng_Rate_1Y_Chg,LA_Tot_Opioid_Clms,LA_Opioid_Prscrbng_Rate,LA_Opioid_Prscrbng_Rate_5Y_Chg,LA_Opioid_Prscrbng_Rate_1Y_Chg
0,2019,National,NaN,National,NaN,Totals,Overall,1239380.0,803549.0,66138200.0,1.501140e+09,4.41,-1.32,-0.27,7290510.0,11.02,-1.73,-0.77
1,2019,State,1.0,Alabama,NaN,Totals,Overall,15575.0,9739.0,1814583.0,2.800821e+07,6.48,-1.40,-0.42,138456.0,7.63,-2.13,-0.73
2,2019,State,2.0,Alaska,NaN,Totals,Overall,2847.0,1935.0,80311.0,1.639104e+06,4.90,-2.01,-0.90,13005.0,16.19,-3.73,-2.42
3,2019,State,4.0,Arizona,NaN,Totals,Overall,25876.0,16578.0,1359955.0,2.569376e+07,5.29,-1.79,-0.33,184250.0,13.55,-2.58,-1.25
4,2019,State,5.0,Arkansas,NaN,Totals,Overall,9700.0,7258.0,998319.0,1.789430e+07,5.58,-1.08,-0.32,76135.0,7.63,-1.69,-0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2015,State,51.0,Virginia,NaN,Totals,Overall,25319.0,19755.0,1720058.0,3.055526e+07,5.63,NaN,-0.29,201649.0,11.72,NaN,0.38
256,2015,State,53.0,Washington,NaN,Totals,Overall,25351.0,19532.0,1658461.0,2.374570e+07,6.98,NaN,-0.19,281071.0,16.95,NaN,-0.20
257,2015,State,54.0,West Virginia,NaN,Totals,Overall,7064.0,5174.0,752812.0,1.288542e+07,5.84,NaN,-0.56,69188.0,9.19,NaN,0.25
258,2015,State,55.0,Wisconsin,NaN,Totals,Overall,20540.0,16177.0,1489660.0,2.539947e+07,5.86,NaN,-0.15,263191.0,17.67,NaN,-0.17


In [6]:
#cdc_df

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
0,AK,2015,April,12 month-ending,"Natural & semi-synthetic opioids, incl. methad...",NaN,100,0.000000,Alaska,Numbers may differ from published reports usin...,**,NaN
1,AK,2015,April,12 month-ending,Cocaine (T40.5),NaN,100,0.000000,Alaska,Numbers may differ from published reports usin...,**,NaN
2,AK,2015,April,12 month-ending,Natural & semi-synthetic opioids (T40.2),NaN,100,0.000000,Alaska,Numbers may differ from published reports usin...,**,NaN
3,AK,2015,April,12 month-ending,Number of Drug Overdose Deaths,126.000000,100,0.000000,Alaska,Numbers may differ from published reports usin...,**,126.0
4,AK,2015,April,12 month-ending,Psychostimulants with abuse potential (T43.6),NaN,100,0.000000,Alaska,Numbers may differ from published reports usin...,**,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44689,YC,2021,May,12 month-ending,"Synthetic opioids, excl. methadone (T40.4)",1792.000000,100,0.298805,New York City,Underreported due to incomplete data.,*,1864.0
44690,YC,2021,May,12 month-ending,Percent with drugs specified,99.230769,100,0.298805,New York City,Underreported due to incomplete data.,*,NaN
44691,YC,2021,May,12 month-ending,Number of Deaths,62248.000000,100,0.298805,New York City,Underreported due to incomplete data.,*,NaN
44692,YC,2021,May,12 month-ending,Cocaine (T40.5),962.000000,100,0.298805,New York City,Underreported due to incomplete data.,*,1005.0


In [9]:
# Dropping years 2020 & 2021 and filtering indicator column 
cdc_year = cdc_df.loc[(cdc_df["Year"] != 2020) & (cdc_df["Year"] != 2021) & (cdc_df["State"] != 'YC')]
cdc_numofdeathsindicator = cdc_year.loc[(cdc_year["Indicator"]=="Number of Deaths")]

# Reset the index values for the dataframes
cdc_numofdeathsindicator = cdc_numofdeathsindicator.reset_index(drop=True)
cdc_numofdeathsindicator

# Changing column names for merging later
cdc_numofdeathsindicator = cdc_numofdeathsindicator.rename(columns={"State" : "State ID", "State Name" : "State"})
cdc_numofdeathsindicator['Data Value'] = pd.to_numeric(cdc_numofdeathsindicator['Data Value'], errors = 'coerce')
cdcnumofdeathsaggregated=cdc_numofdeathsindicator.groupby(['State ID', 'Year']).agg({'Data Value' : ['sum']})

cdcnumofdeathsaggregated


Data Value
                     sum
State ID Year           
AK       2015    50086.0
         2016    51141.0
         2017    51597.0
         2018    51865.0
         2019    53484.0
...                  ...
WY       2015    53812.0
         2016    52723.0
         2017    52181.0
         2018    54073.0
         2019    56144.0

[260 rows x 1 columns]

In [11]:
# Dropping years 2020 & 2021 and filtering indicator column 
cdc_year = cdc_df.loc[(cdc_df["Year"] != 2020) & (cdc_df["Year"] != 2021) & (cdc_df["State"] != 'YC')]
cdc_numofdrugdeathsindicator = cdc_year.loc[(cdc_year["Indicator"]=="Number of Drug Overdose Deaths")]

# Reset the index values for the dataframes
cdc_numofdrugdeathsindicator = cdc_numofdrugdeathsindicator.reset_index(drop=True)
cdc_numofdrugdeathsindicator

# Changing column names for merging later
cdc_numofdrugdeathsindicator = cdc_numofdrugdeathsindicator.rename(columns={"State" : "State ID", "State Name" : "State"})
cdc_numofdrugdeathsindicator['Data Value'] = pd.to_numeric(cdc_numofdrugdeathsindicator['Data Value'], errors = 'coerce')
cdc_numofdrugdeathsaggregated=cdc_numofdrugdeathsindicator.groupby(['State ID', 'Year']).agg({'Data Value' : ['sum']})

cdc_numofdrugdeathsaggregated


Data Value
                     sum
State ID Year           
AK       2015     1472.0
         2016     1594.0
         2017     1571.0
         2018     1442.0
         2019     1544.0
...                  ...
WY       2015     1229.0
         2016     1138.0
         2017      871.0
         2018      763.0
         2019      797.0

[260 rows x 1 columns]

In [12]:
# Dropping years 2020 & 2021 and filtering indicator column 
cdc_year = cdc_df.loc[(cdc_df["Year"] != 2020) & (cdc_df["Year"] != 2021) & (cdc_df["State"] != 'YC')]
cdc_numofopioiddeathsindicator = cdc_year.loc[(cdc_year["Indicator"]=="Opioids (T40.0-T40.4,T40.6)")]

# Reset the index values for the dataframes
cdc_numofopioiddeathsindicator = cdc_numofopioiddeathsindicator.reset_index(drop=True)
cdc_numofopioiddeathsindicator
                                                                             
                                                                             
# Changing column names for merging later
cdc_numofopioiddeathsindicator = cdc_numofopioiddeathsindicator.rename(columns={"State" : "State ID", "State Name" : "State"})
cdc_numofopioiddeathsindicator['Data Value'] = pd.to_numeric(cdc_numofopioiddeathsindicator['Data Value'], errors = 'coerce')
cdc_numofopioiddeathsaggregated=cdc_numofopioiddeathsindicator.groupby(['State ID', 'Year']).agg({'Data Value' : ['sum']})

cdc_numofopioiddeathsaggregated                                                                           
                                                                             

Data Value
                     sum
State ID Year           
AK       2015      165.0
         2016     1163.0
         2017     1110.0
         2018      990.0
         2019      962.0
...                  ...
WY       2015        0.0
         2016        0.0
         2017      404.0
         2018      497.0
         2019      508.0

[225 rows x 1 columns]

In [13]:
# Merging Medicaid & Medicare data with an Outer Join
insurance = pd.merge(care_clean, caid_clean, how = "outer", on = ["Year", "State"])
insurance

,Year,Prscrbr_Geo_Lvl,Prscrbr_Geo_Cd,State,RUCA_Cd,Breakout_Type,Breakout,Tot_Prscrbrs,Tot_Opioid_Prscrbrs,Tot_Opioid_Clms_x,...,Plan_Type,Tot_Opioid_Clms_y,Tot_Clms_y,Opioid_Prscrbng_Rate_y,Opioid_Prscrbng_Rate_5Y_Chg_y,Opioid_Prscrbng_Rate_1Y_Chg_y,LA_Tot_Opioid_Clms_y,LA_Opioid_Prscrbng_Rate_y,LA_Opioid_Prscrbng_Rate_5Y_Chg_y,LA_Opioid_Prscrbng_Rate_1Y_Chg_y
0,2019,National,NaN,National,NaN,Totals,Overall,1239380.0,803549.0,66138200.0,...,All,21978286.0,677248025.0,3.25,-3.22,-0.44,3108845.0,14.15,6.67,4.62
1,2019,State,1.0,Alabama,NaN,Totals,Overall,15575.0,9739.0,1814583.0,...,All,224310.0,7230251.0,3.10,-3.23,-1.08,10308.0,4.60,0.16,0.48
2,2019,State,2.0,Alaska,NaN,Totals,Overall,2847.0,1935.0,80311.0,...,All,65394.0,1434139.0,4.56,-3.33,-1.20,8268.0,12.64,0.88,-0.71
3,2019,State,4.0,Arizona,NaN,Totals,Overall,25876.0,16578.0,1359955.0,...,All,595206.0,14923326.0,3.99,-5.14,-0.75,44311.0,7.44,-2.79,-0.52
4,2019,State,5.0,Arkansas,NaN,Totals,Overall,9700.0,7258.0,998319.0,...,All,216081.0,4999626.0,4.32,-3.12,-0.88,8129.0,3.76,-0.34,-1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2015,State,51.0,Virginia,NaN,Totals,Overall,25319.0,19755.0,1720058.0,...,All,583116.0,10186332.0,5.72,NaN,-0.62,42311.0,7.26,NaN,-0.26
256,2015,State,53.0,Washington,NaN,Totals,Overall,25351.0,19532.0,1658461.0,...,All,1136474.0,14016747.0,8.11,NaN,-0.89,106143.0,9.34,NaN,0.18
257,2015,State,54.0,West Virginia,NaN,Totals,Overall,7064.0,5174.0,752812.0,...,All,520370.0,9293729.0,5.60,NaN,-1.30,25575.0,4.91,NaN,0.66
258,2015,State,55.0,Wisconsin,NaN,Totals,Overall,20540.0,16177.0,1489660.0,...,All,962286.0,11306189.0,8.51,NaN,-0.35,146248.0,15.20,NaN,1.04


In [14]:
# Merging CDC data & Insurace Data w/ an outer join and on Year and State
#data = pd.merge(cdcnumofdeathsaggregated, insurance, how = "outer", on = ["Year", "State"])
#data

C:\Users\CHRIST~1\AppData\Local\Temp/ipykernel_17172/2310095091.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  data = pd.merge(cdcnumofdeathsaggregated, insurance, how = "outer", on = ["Year", "State"])


KeyError: 'State'